In [5]:
!pip install keras

In [2]:
#https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-language-model-nlp-python-code/

In [1]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
#from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import pad_sequences

In [2]:
data_text = ""


for iPath in ['trainSmallQa.txt', 'trainSmallWorld0.txt']:#, 'trainSmallMath0.txt', 'trainEnToNarsese0.txt', 'trainMisc0.txt', 'trainSmallPhysics0.txt', 'trainFullWiki0.txt', 'trainMisc1.txt']:
    with open(iPath, 'r') as file:
        content = file.read()
        data_text = data_text+content




import re

def text_cleaner(text):
    # lower case text
    text2 = text.lower()
    #text2 = re.sub(r"'s\b","", text2)
    # remove punctuations
    #text2 = re.sub("[^a-zA-Z]", " ", text2) 
    #print(text2) # DBG
    
    long_words=[]
    #
    # remove short word
    for i in text2.split():
        if len(i)>=2:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()
    #return text2

# preprocess the text
data_new = text_cleaner(data_text)


# store cleaned text for debugging
if True: # code block
    f = open('z_db_preprocessedDump.txt', 'w')
    f.write(data_new)
    f.close()





def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

# create sequences   
sequences = create_seq(data_new)





# create a character mapping index
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences

# encode the sequences
sequences = encode_seq(sequences)










# predict

import numpy as np
#from keras.preprocessing.sequence import pad_sequences

def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict character probabilities
        yhat = model.predict(encoded, verbose=0)
        # get the index of the highest probability character
        yhat = np.argmax(yhat)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

Total Sequences: 25204


In [3]:

from sklearn.model_selection import train_test_split

# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.06, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (23691, 30) Val shape: (1513, 30)


In [5]:
import keras

# define model
model = Sequential()
model.add(Embedding(vocab, 70, input_length=30, trainable=True)) # 50
#model.add(Dense(440, activation='relu')) # 0     # 50
#model.add(GRU(400, return_sequences=False, recurrent_dropout=0.1, dropout=0.1)) # 200    # 185 # 170 # 150

s0 = 1.0 # 70'000parameters
s0 = 1.5 # 70'000parameters
#s0 = 1.5*3.0*1.2
s0 = 0.8 # 47'000 parameters for 7800 samples
s0 = 1.5
s0 = 1.75
model.add(Dense(int(124*s0*1.5), activation='relu')) # 0     # 50
model.add(GRU(int(80*s0), return_sequences=False, recurrent_dropout=0.1, dropout=0.1)) # 200    # 185 # 170 # 150


model.add(Dense(vocab, activation='softmax'))
print(model.summary())

# compile the model
learning_rate = 0.0008 # 0.0005
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer=optimizer)



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 30, 70)            5110      
                                                                 
 dense_2 (Dense)             (None, 30, 325)           23075     
                                                                 
 gru_1 (GRU)                 (None, 140)               196140    
                                                                 
 dense_3 (Dense)             (None, 73)                10293     
                                                                 
Total params: 234,618
Trainable params: 234,618
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
if False:
    import keras
    
    print('loading...')
    model = keras.models.load_model('lm87A.keras.model')
    print('loaded')


In [6]:
# module which is used to decide when the search for a better network is aborted based on last epoch of improvement in test-set
class CountDown(object):
    def __init__(self, countdownVal):
        self.countdownVal = countdownVal
        self.v = -self.countdownVal # counter
    
    def reset(self):
        self.v = -self.countdownVal
    
    def count(self):
        self.v += 1
    
    def check(self):
        return self.v >= 0

    

In [ ]:
# Define a checkpoint to save the model with the lowest validation loss
checkpoint = ModelCheckpoint('lm87A_lowestValidationLoss.keras.model', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

abortCountdown = CountDown(130) # give up after 13 of epochs of no improvement
bestValidationsetLoss = 1e10

# fit the model
for it in range(int(250/1)):
    for z in range(4):
        print('')
    
    
    print(f"epoch={it*1}")
    
    model.fit(X_tr, y_tr, epochs=1, verbose=2, validation_data=(X_val, y_val), callbacks=[keras.callbacks.ProgbarLogger(), checkpoint])
    model.save('lm87A.keras.model')
    
    
    # Evaluate the model on the validation set and print the loss and accuracy
    loss, acc = model.evaluate(X_val, y_val)
    print(f'Validation loss={loss:.4f}, Validation accuracy={acc:.4f}')
    
    
    if True: # codeblock
        abortCountdown.count()
        if loss < bestValidationsetLoss:
            bestValidationsetLoss = loss
            abortCountdown.reset() # reset counter because it's better

        if abortCountdown.check(): # is countdown fullfilled
            # this means that the countdown is over, and this means that we didn't find a better model for a long time
            print('info: didn\'t find a better model for a to long time ... give up')
            break
    
    
    
    
    
    #s0 = generate_seq(model, mapping, 30, "q: what is a llama?", 60)
    #print(f'sample={s0}')
    
    #s0 = generate_seq(model, mapping, 30, "q: what is a neutron?", 60)
    #print(f'sample={s0}')
    
    s0 = generate_seq(model, mapping, 30, "exponentiation operation ", 60)
    print(f'sample={s0}')
    
    s0 = generate_seq(model, mapping, 30, "sine and cosine are trigonometric functions angle", 80)
    print(f'sample={s0}')

    s0 = generate_seq(model, mapping, 30, "the oldest definitions trigonometric functions", 80)
    print(f'sample={s0}')

    
    s0 = generate_seq(model, mapping, 30, "real number number that can", 80)
    print(f'sample={s0}')
    
    
    

    s0 = generate_seq(model, mapping, 30, "extract relations", 80)
    print(f'sample={s0}')
        
    s0 = generate_seq(model, mapping, 30, "water", 80)
    print(f'sample={s0}')
    
    for iQueryInner in ['food', 'llama', 'google', 'frog', 'llgic']:
        s0 = generate_seq(model, mapping, 30, f"q: what is {iQueryInner}? ", 80)
        print(f'sample={s0}')
    
    
    






epoch=0
741/741 - 60s - loss: 2.5178 - acc: 0.2884 - val_loss: 2.2153 - val_acc: 0.3437 - 60s/epoch - 81ms/sample

Epoch 1: val_loss improved from inf to 2.21529, saving model to lm87A_lowestValidationLoss.keras.model
INFO:tensorflow:Assets written to: lm87A_lowestValidationLoss.keras.model/assets
INFO:tensorflow:Assets written to: lm87A.keras.model/assets
48/48 [==============================] - 0s 10ms/step - loss: 2.2153 - acc: 0.3437
Validation loss=2.2153, Validation accuracy=0.3437
sample=exponentiation operation is and and and and and and and and and and and and and and a
sample=sine and cosine are trigonometric functions angle and and and and and and and and and and and and and and and and and and and and
sample=the oldest definitions trigonometric functions and and and and and and and and and and and and and and and and and and and and
sample=real number number that cand is and and and and and and and and and and and and and and and and and and and
sample=extract relations

In [ ]:
mapping
0